In [1]:
from supabase import create_client, Client
from datetime import datetime, timezone
import time
import openai
import tiktoken
import requests
import json
from PIL import Image
from io import BytesIO
from dataplane import s3_upload
import boto3
from botocore.client import Config
import subprocess
import random
import os
import re
from pydub import AudioSegment
from datetime import datetime, timedelta
import math
from supabase.lib.client_options import ClientOptions

from sentence_transformers import SentenceTransformer
from transformers import pipeline
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
import json

qdrant_client = QdrantClient(
    "https://6322e625-6ce9-440b-9a37-fa2fe5fd85e1.europe-west3-0.gcp.cloud.qdrant.io",
    api_key="KIQ65rYvlGD51tYKfH42dwPN40XS_-n0ux9qBeMiYLCvPqWH0keZZw",
)


clientOptions = ClientOptions(postgrest_client_timeout=999999)

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# supabase: Client = create_client("http://supa-supabase-kong.supabase.svc.cluster.local:8000", "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.ewogICJyb2xlIjogInNlcnZpY2Vfcm9sZSIsCiAgImlzcyI6ICJzdXBhYmFzZSIsCiAgImlhdCI6IDE3MTM4MTA2MDAsCiAgImV4cCI6IDE4NzE1NzcwMDAKfQ.wdLRO9uTwoZg-tdiP_yusL7a4Vx5_kO2edcWq-PtZu0")
supabase: Client = create_client("https://s.a.synopseai.com", 
                                 "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.ewogICJyb2xlIjogInNlcnZpY2Vfcm9sZSIsCiAgImlzcyI6ICJzdXBhYmFzZSIsCiAgImlhdCI6IDE3MTM4MTA2MDAsCiAgImV4cCI6IDE4NzE1NzcwMDAKfQ.wdLRO9uTwoZg-tdiP_yusL7a4Vx5_kO2edcWq-PtZu0",
                                 clientOptions)


In [7]:

qdrant_client.create_collection(
    collection_name="synopse_ta_1",
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE, on_disk=True),
    hnsw_config=models.HnswConfigDiff(on_disk=True),
)

True

In [8]:
i = -1 
while True:
    i += 1
    data2 = supabase.table('t_f01_articles_vector_ta').select('embedding, article_id, created_at').limit(100).offset(100*i).order('article_id').execute()
    if (len(data2.data) == 0):
        break
    for item2 in data2.data:
        embedding_list = json.loads(item2['embedding'])
        qdrant_client.upsert(
            collection_name="synopse_ta_1",
            points=[PointStruct(vector=embedding_list, id=item2['article_id'])],
        )

ResponseHandlingException: The read operation timed out

In [ ]:
i = -1 
while True:
    i += 1
    data2 = supabase.table('t_f01_articles_vector_te').select('embedding, article_id, created_at').limit(100).offset(100*i).order('article_id').execute()
    if (len(data2.data) == 0):
        break
    for item2 in data2.data:
        embedding_list = json.loads(item2['embedding'])
        qdrant_client.upsert(
            collection_name="synopse_te_1",
            points=[PointStruct(vector=embedding_list, id=item2['article_id'])],
        )

In [ ]:

qdrant_client.create_collection(
    collection_name="synopse_hi_1",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE, on_disk=True),
    hnsw_config=models.HnswConfigDiff(on_disk=True),
)

In [4]:
print(len([-0.01672423,-0.007596908,-0.023405422,-0.0041584754,0.011128731,0.018345699,0.017923484,0.0035754263,-0.0004732839,0.00976857,0.009624479,-0.0046650684,-0.007844014,-0.006868145,-0.007049717,-0.02783043,0.0115904845,-0.012740495,-0.00093056494,-0.0045007356,-0.016744299,-0.024199523,-0.022843348,0.012281612,-0.006229686,0.0016434143,0.0027212468,0.008905398,-0.013384035,0.021558568,-0.0073797475,-0.0022994324,-0.012436895,0.00301933,-0.0128624765,0.0061484557,0.018318713,0.0065566786,-0.00074958196,0.00031550534,0.0064005004,0.0075933724,-0.01177203,-0.0011744616,-0.017824931,-0.007978061,-0.016864808,-0.00922588,0.027558055,0.014668849,-0.0122255795,-0.009817931,-0.011084152,-0.008060439,-0.0070447274,-0.011962424,-0.012136313,0.013382721,-0.008619608,0.0025453644,-0.009029854,0.012412977,0.00809016,-0.012934446,-0.0057142847,-0.0012053673,-0.010325029,-0.003332356,0.0014544525,-0.019549532,-0.0034307025,-0.0049361023,-0.01542767,-0.0031365468,-0.022422008,-0.03874547,0.017275115,-0.0045241746,-0.005415646,0.007908696,-0.0045264116,0.006770158,0.0062994123,-0.015630648,0.0048652757,0.0062196446,-0.0031214845,-0.023549976,-0.015451968,-0.0029202723,0.005068895,-0.0067381696,-0.010987802,0.0045116027,-0.015175965,0.017241893,0.0124833835,0.0005658688,0.0029241198,0.011651026,-0.024316626,-0.014991001,0.014482325,0.0026059442,-0.003705413,-0.0040764753,9.5495925e-06,0.0067982227,0.023078889,0.0055378885,0.0007272534,0.013203368,0.011385241,0.02661792,0.006614392,0.0118807815,0.015501615,0.013708888,0.031385846,-0.017818155,0.03567146,-0.015676184,-0.020760754,-0.015745094,0.003311236,-0.015726076,-0.025845166,-0.0104304105,0.0147608705,-0.010642455,-0.01947286,-0.017248005,0.005094705,-0.00079800125,0.0017937045,0.004527226,0.003268763,-0.011649692,-0.025180107,0.0033828712,-0.024551054,-0.012257204,-0.006011095,-0.014243704,0.004195801,-0.004407807,0.030878859,0.00982332,-0.009675763,0.0008739242,-0.026877731,0.02208592,-0.007775457,0.011677495,0.00032583685,-0.012790266,-0.017693361,-0.019923823,0.0024441322,-0.0073718736,-0.011416388,0.014140383,-0.020337125,0.028563935,0.014397188,0.021948243,0.018940136,0.0056064306,0.0056357817,-0.013142074,0.012642871,0.0005102108,-0.0017738353,0.023645518,-0.0050989264,-0.003936234,0.014350136,-0.004242455,0.013879634,-0.013682205,-0.039705303,0.025222171,-0.012837365,-0.031105751,-0.00841118,0.010408455,-0.0123927845,-0.00079115795,-0.07083271,0.026448254,0.03288344,-0.036411006,-1.5814621e-05,0.0043272837,-0.0022393698,-0.013409636,0.007866614,0.016423954,0.010803584,-0.00078909827,-0.0011611071,-0.0032878406,-0.02541754,-0.025830075,-0.006268024,-0.028135048,-0.0074613304,-0.01647452,0.0042152884,-0.0031350683,-0.006718827,0.023431594,0.012621677,-0.0032689131,0.019615455,-0.02055894,-0.014174868,0.0038479173,-0.009022777,0.02277349,0.009340934,0.029084686,-0.00079884334,0.004933267,-0.00039498336,-0.004513225,0.0349909,-0.01701512,-0.020902194,-0.0044996426,-0.004540708,0.006814607,-0.005765832,0.006906089,0.009295375,0.0050848117,0.0029125647,0.016731806,-0.0018077523,0.0026840826,-0.007716415,0.0042845616,0.003987626,0.027739165,0.0026483699,0.0048968275,-0.0045233336,0.008957495,0.001544189,-0.011490446,0.0053542578,-0.0057194256,0.033231024,-0.01804426,-0.021948298,-0.011563063,0.004575084,0.04628166,0.020194745,-0.015802063,0.009283608,-0.01245533,0.011706615,-0.014415075,-0.006019488,0.011618365,0.009585638,-0.011362183,-0.030283665,0.004921697,0.005814136,0.043676194,-0.0007789035,-0.01647506,-0.00028438162,-0.0065114223,0.028803797,0.009893696,-0.019945644,-0.022107942,-0.027934272,-0.004378144,-0.017564096,0.0004015845,0.0071076383,-0.007667906,-7.422283e-05,-0.0148446765,-0.0067737303,0.0074693896,0.016972391,-0.022831032,-0.015108256,0.015826311,0.029203655,0.014759055,-0.018763265,-0.013832559,0.025403462,-0.013013992,-0.014642857,0.022448037,-0.010461442,0.0009846432,0.0028351254,0.0010444717,0.015322116,0.024020435,-0.01285855,0.006285498,-0.012092107,0.0020563586,0.02115659,-0.01979888,-0.035247944,0.00023220516,-0.013198076,-0.017021084,-0.010179455,0.016835079,-0.01396173,-0.022147663,0.0017118911,0.01218532,0.009506014,-0.006450054,-0.043817014,-0.0052577252,0.03673506,-0.026521137,0.017826624,-0.011455946,-0.011432516,0.006200384,-0.012487068,-0.0057585407,0.005694847,0.01996824,0.0011969,0.02256928,0.034764443,0.015641391,0.022196848,-0.018889824,0.016878292,-0.0068850843,0.015123183,0.0038303134,-0.020315256,0.011147812,0.019684985,-0.0024514627,0.0077418257,-0.013624616,-0.01443713,0.00630684,0.0104197925,-0.011784052,0.011665289,0.002850675,-0.016731804,0.011351186,0.006384156,-0.0030161454,0.0033926978,0.0011386707,0.025798561,-0.0047805556,-0.006109958,-0.01945006,0.013118719,-0.026265455,-0.014950512,0.03138097,0.031951353,0.018896349,0.014523254,-0.0022757614,0.022085845,0.0035401438,-0.0043423544,0.010835046,-0.012201173,0.009578925,-0.020324584,0.014117587,-0.0016804449,-0.0070348224,-0.0029847824,0.0010074101,-0.011917908,0.005385953,0.025747666,0.021129588,0.0067121675,0.0112082455,0.019848382,-0.02221349,0.027603827,-0.001622846,0.033866797,-0.0050754077,-0.00890157,0.020527035,0.024098368,-0.006068511,-0.018867355,0.016045913,0.000861124,0.011526453,-4.3276666e-26,-0.0023280806,0.0019187883,0.00928403,0.022949064,0.022374038,0.022922246,-0.0031927524,-0.003047054,0.026468948,-0.0028533493,-0.011170509,0.0003046884,-0.0042840466,-0.017073808,0.004197375,0.0076723713,-0.006047811,0.016043013,0.015071753,0.009191013,-0.014991129,0.0057555754,-0.014882626,0.0016330067,0.019656092,-0.0064024315,0.009348829,0.0023130146,0.009724612,-0.024391644,-0.016457355,-0.012273687,0.0077898134,0.0030901346,-6.496834e-05,-0.021795144,-0.032168645,-0.01036796,-0.025794158,-0.0001660517,-0.007474817,-0.021938765,0.049932327,0.0056765405,0.001607286,-0.002176664,-0.01259741,-0.016965082,-0.0153866,0.014852112,0.011165975,-0.019705994,-0.002508724,0.01142807,0.015969813,-0.0016306802,-0.020365598,0.008392126,-0.0071575427,0.006947732,0.016632745,0.0008109175,-0.009089261,-0.016304126,0.020614212,0.004477072,-0.007123301,0.015100302,0.0011925909,0.010042575,8.633769e-09,0.019049302,0.0027856089,-0.028387086,0.011834976,-0.010581349,-0.007123721,0.019648544,-0.015476229,0.019453654,0.016398702,-0.014801031,0.006990037,-0.026523564,0.015436163,0.0118600875,-0.014072544,-0.0029344603,0.01801127,-0.004845912,0.019782292,-0.017621664,-0.0029053863,-0.006698142,0.014734175,0.0031861616,0.0045639975,0.007819705,0.0010432398,0.020000009,0.007943631,0.030124769,-0.009378523,0.010558149,0.009078345,-0.027879113,-0.0077643245,-0.016585529,-0.0009048942,0.013408563,0.039754923,-0.011567307,-0.010718078,0.0017368061,-0.006048111,0.04196337,-0.023367548,-0.0015176117,0.009146896,-0.010450954,-0.017966487,0.02544216,0.001539229,-0.025263244,0.008667822,0.0068620923,-0.015174415,-0.009178179,-0.01802711,-0.010191948,-0.0024756754,0.00286601,-0.020023735,-0.047069144,-0.0051458445,0.004223322,-0.005623194,0.0024849488,-0.0033226025,-0.013568055,0.00122281,0.016698947,-0.0016749589,0.0017673138,0.034320213,0.012531767,0.0056636534,-0.0029627178,-0.008015583,-0.009801884,-0.010563982,-0.012351299,-0.018547101,-0.002677505,0.012969227,0.021617267,-0.0114499405,-0.0038235164,0.010148369,-0.010004999,-0.013673919,-0.005964637,0.0062893312,-0.008680532,0.02027422,0.0060061384,-0.019306289,0.022770679,0.003171926,0.011126232,0.049732983,-0.010925886,-0.0070115174,0.022851301,0.007472912,-0.009249298,-0.0063408082,0.00847511,0.01401669,0.013414591,-0.0028638423,0.0029564337,0.011073824,-0.01017991,-0.00535014,-0.0094141895,0.006065013,-0.0033444308,-0.020200234,0.00992543,0.035196126,-0.016458899,-0.0030206228,-0.021823494,-0.020705104,-0.004889226,-0.026706938,0.0123978425,-0.017345645,0.0014793904,0.003423874,-0.015768344,-0.0082219755,0.0034286566,0.019930108,0.026818516,-0.014495762,-0.00027064217,-0.0036814872,0.0003175051,-0.019535545,0.0015473386,-0.016720353,0.014789577,-0.021088738,0.014934593,-0.021286098,0.029452926,0.013286599,0.0073292055,0.012353605,0.025591847,0.0005591936,0.0018345234,0.009216312,0.008113725,0.031274386,-0.021457143,-0.020828549,0.01324119,0.0049771457,-0.029166184,-0.005069331,0.0012681832,0.031862225,0.0029470094,0.015044886,-0.003609912,-5.975193e-05,-0.0059311953,-0.023934215,-0.015533522,0.002895065,-0.03784563,-0.009946129,0.033998523,-0.005049818,-0.017714025,0.027442578,0.002388455,-0.0008487498,-0.00927131,0.0038139008,-0.0013128999,-0.0044469703,-0.013365979,0.012172851,-0.012257637,0.0011001526,0.0028102812,-0.018083138,-0.015626999,-0.0035512412,0.01872165,-0.018417338,-0.026492914,0.012790413,-0.008243656,-0.0036469372,0.0005824779,0.0043248977,0.008755227,-0.015174113,0.018897228,0.014552963,-0.03204774,-0.019227346,0.00517387,0.006929709,0.020522907,0.03221816,0.00544216,-0.004430314,-0.0067458153,0.009343766,-0.009808789,-0.014016488,0.012345358,-0.0043557645,-0.009514065,0.02557706,0.01692192,-0.007735216,0.005309338,0.012447373,-0.0030910303,0.008060348,0.011142002,-0.011839254,0.016009746,-0.0027957635,0.003358738,0.008868554,0.007467107,-0.014032108,-0.02446415,0.0014666286,-0.0021010689,-0.0032033552,-0.002475313,-0.0006248739,0.000610743,0.0026221348,-0.0052890023,0.017210063,0.0038593372,0.012102702,0.017067013,-0.005453847,0.012798358,-0.01789838,-0.0059861983,0.00011615891,-6.6780244e-06,-0.026826054,0.015438689,0.0071176756,0.004347014,0.0102257915,-0.006993057,-3.9955503e-05,-0.029023327,0.009352389,0.017348003,0.003918145,0.008903744,0.0077618817,-0.010583005,-0.0027978704,0.00010125539,0.003582807,-0.0012002598,0.0012044054,0.016784048,-0.016905699,-0.013250556,0.012284916,0.015738975,0.00617067,0.012336679,0.005897412,0.004200478,0.023421777,0.0016841139,-0.023385696,0.007814485,-0.0014418734]))

768
